In [1]:
import pandas as pd
import numpy as np
from utils import *
import os
import fitz
from pdf2image import convert_from_path
import pytesseract
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pdfplumber

#### Step 1: Change Working Directory

In [2]:
os.chdir('/Users/sooyeonkim/Desktop/TenStrands')

In [3]:
master = pd.read_excel('District Level Master Spreadsheet-UCB Policy Analysis - BP_AR 3514.xlsx')

In [4]:
master['AR3514 PDF Name'] = master['District Name'].str.replace(' ', '_') + '_' + master['County'].str.replace(' ', '_') + '_AR3514.pdf'

master['BP3514 PDF Name'] = master['District Name'].str.replace(' ', '_') + '_' + master['County'].str.replace(' ', '_') + '_BP3514.pdf'

master.to_excel('District Level Master Spreadsheet-UCB Policy Analysis - BP_AR 3514.xlsx', index=False)

#### Step 2: Load Data

In [5]:
# load  dataset that contains paths to PDFs
df = master

# initialize columns to store PDF text
df['BP3514: PDF Text'] = np.nan
df['AR3514: PDF Text'] = np.nan

# set the columns to object type explicitly to store strings
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].astype('object')
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].astype('object')

In [6]:
# base paths
base_path = "/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/BP_AR_3514/"

# constructing full paths
master['AR3514: Path to PDF'] = base_path + master['AR3514'].astype(str)
master['BP3514: Path to PDF'] = base_path + master['BP3514'].astype(str)

In [7]:
def extract_text_from_pdf(file_path):
    """Extracts text from a PDF file."""
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ''.join([page.extract_text() or '' for page in pdf.pages])
        return text
    except Exception as e:
        print(f"Failed to process {file_path}: {e}")
        return None  

In [8]:
for index, row in df.iterrows():
    if pd.notna(row['AR3514: Path to PDF']):
        ar_text = extract_text_from_pdf(row['AR3514: Path to PDF'])
        df.at[index, 'AR3514: PDF Text'] = ar_text

    if pd.notna(row['BP3514: Path to PDF']):
        bp_text = extract_text_from_pdf(row['BP3514: Path to PDF'])
        df.at[index, 'BP3514: PDF Text'] = bp_text

Failed to process /Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/BP_AR_3514/Albany_City_Unified_Alameda_BP3514.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/BP_AR_3514/Albany_City_Unified_Alameda_BP3514.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/BP_AR_3514/Fremont_Unified_Alameda_AR3514.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/BP_AR_3514/Fremont_Unified_Alameda_AR3514.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/BP_AR_3514/Fremont_Unified_Alameda_BP3514.pdf: [Errno 2] No such file or directory: '/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/BP_AR_3514/Fremont_Unified_Alameda_BP3514.pdf'
Failed to process /Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/data/BP_AR_3514/New_Haven_Unified_Alameda_AR3514.pdf: [Errno

In [9]:
df[df['BP3514: PDF Text'].isna() & (df['BP: 3514 Environmental Safety'] == 1)]['CDS Code']

4       1611760000000
56      9619290000000
64     10621170000000
73     10624140000000
79     10739990000000
83     10767780000000
108    13630810000000
121    14766870000000
142    15637500000000
174    18641390000000
221    19648810000000
266    22655320000000
337    30666130000000
385    34752830000000
400    36677020000000
403    36677770000000
410    36678680000000
413    36679180000000
415    36679590000000
425    37680070000000
428    37680560000000
434    37681300000000
458    38684780000000
461    39685690000000
464    39685930000000
508    42692600000000
518    43694270000000
533    43696410000000
539    44697320000000
589    49707220000000
633    51714230000000
650    54718600000000
654    54719930000000
662    54721990000000
663    54722070000000
674    55723480000000
685    56725120000000
686    56725200000000
694    56738740000000
Name: CDS Code, dtype: int64

In [10]:
df[df['AR3514: PDF Text'].isna() & (df['AR: 3514 Environmental Safety Regulation'] == 1)]['CDS Code']

4       1611760000000
30      6616060000000
37      7617050000000
57      9619520000000
79     10739990000000
80     10752340000000
100    12630320000000
110    13631070000000
174    18641390000000
215    19648080000000
221    19648810000000
255    20764140000000
257    21653910000000
274    23739160000000
300    27660680000000
382    34674390000000
385    34752830000000
400    36677020000000
413    36679180000000
417    36738900000000
425    37680070000000
428    37680560000000
434    37681300000000
444    37683380000000
446    37683610000000
464    39685930000000
467    39686760000000
503    42692030000000
518    43694270000000
531    43696170000000
589    49707220000000
654    54719930000000
663    54722070000000
674    55723480000000
Name: CDS Code, dtype: int64

In [11]:
def clean_text(text):
    """Remove or replace illegal XML characters from the text."""
    if text is not None and isinstance(text, str):
        # XML 1.0 valid characters:
        illegal_xml_chars = re.compile(u'[\u0000-\u0008\u000B\u000C\u000E-\u001F\uD800-\uDFFF\uFFFE\uFFFF]')
        return illegal_xml_chars.sub('', text)
    return text

In [12]:
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].apply(clean_text)
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].apply(clean_text)

In [13]:
def clean_pdf_text(text):
    text = text.lower() # Lowercase text 
    text = re.sub(r'boardpolicymanual', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\w+unifiedschooldistrict\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b\d{6,}\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'policy reference disclaimer.*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'supporting documents.*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'state references.*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'education code.*$', '', text, flags=re.IGNORECASE)
    text = re.sub(r'legal reference.*', '', text, flags=re.IGNORECASE | re.DOTALL) # regex to remove 'legal reference' and everything after


    tokens = word_tokenize(text) # Tokenize text
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens if re.sub(r'[^\w\s]', '', token)] # regex to match punctuation
    stop_words = set(stopwords.words('english')) # stop words
    tokens = [word for word in tokens if word not in stop_words] # removing stop words 
    lemmatizer = WordNetLemmatizer() 
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # lemmetize 
    cleaned_text = ' '.join(tokens) # join tokens back into a string
    
    return cleaned_text

In [14]:
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].astype(str)
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].astype(str)

df['BP3514: PDF Text'] = df['BP3514: PDF Text'].apply(clean_pdf_text)
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].apply(clean_pdf_text)

In [15]:
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].str.replace('ppoolliiccyy eennvviirroonnmmeennttaall ssaaffeettyy ssttaattuuss adopted oorriiggiinnaall aaddoopptteedd ddaattee', '', regex=False)
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].str.replace('llaasstt rreevviieewweedd ddaattee', '', regex=False)
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].str.replace(r'\b\d{2,}\b', '', regex=True)
df['BP3514: PDF Text'] = df['BP3514: PDF Text'].str.replace(r'\b\w+schooldistrict\b', '', regex=True)

In [21]:
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].str.replace('rreegguullaattiioonn eennvviirroonnmmeennttaall ssaaffeettyy ssttaattuuss adopted oorriiggiinnaall aaddoopptteedd ddaattee', '', regex=False)
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].str.replace('llaasstt rreevviisseedd ddaattee', '', regex=False)
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].str.replace('llaasstt rreevviieewweedd ddaattee', '', regex=False)
df['AR3514: PDF Text'] = df['AR3514: PDF Text'].str.replace(r'\b\d{5,}\b', '', regex=True)


In [23]:
df = df.drop(columns = ['Unnamed: 40', 'AR3514', 'BP3514'])

KeyError: "['Unnamed: 40', 'AR3514', 'BP3514'] not found in axis"

In [24]:
df

,CDS Code,County,District Name,Locale Classifications,District Type,2023-24 Student Enrollment,2023-24 % Unduplicated,BP: 3514 Environmental Safety,BP 3514 Year Adopted,BP 3514 Year of Policy Update if applicable,...,"Plans, Communication, and Awards Indicators (up to 3)",District-Wide Campus Sustainability Indicators (up to 3),District-Wide Environmental Community and Culture Indicators (up to 3),Environmental and Climate Action Score (20 points),BP3514: Path to PDF,BP3514: PDF Text,AR3514: Path to PDF,AR3514: PDF Text,AR3514 PDF Name,BP3514 PDF Name
0,1611190000000,Alameda,Alameda Unified,Suburban,Unified School District,10557,41,1,2009,2018,...,2.5,3.00,1.5,14.00,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,llaasstt rreevviisseedd ddaattee governin...,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,superintendent may designate train one e...,Alameda_Unified_Alameda_AR3514.pdf,Alameda_Unified_Alameda_BP3514.pdf
1,1611270000000,Alameda,Albany City Unified,Suburban,Unified School District,3574,37,0,NaN,NaN,...,1.5,3.00,2.0,15.00,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,none,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,superintendent may designate train one e...,Albany_City_Unified_Alameda_AR3514.pdf,Albany_City_Unified_Alameda_BP3514.pdf
2,1611500000000,Alameda,Castro Valley Unified,Suburban,Unified School District,9489,43,1,2018,*,...,1.0,3.00,1.5,10.50,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,boarddocs plus book board policy section bu...,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,1125 638 pm boarddocs plus book board policy s...,Castro_Valley_Unified_Alameda_AR3514.pdf,Castro_Valley_Unified_Alameda_BP3514.pdf
3,1611680000000,Alameda,Emery Unified,Suburban,Unified School District,600,81,1,2004,2004,...,0.0,2.50,0.5,6.00,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,board education recognizes obligation prov...,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,air quality superintendent designee shall ...,Emery_Unified_Alameda_AR3514.pdf,Emery_Unified_Alameda_BP3514.pdf
4,1611760000000,Alameda,Fremont Unified,Suburban,Unified School District,33063,33,1,2020,2020,...,1.5,3.00,2.5,14.00,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,none,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,none,Fremont_Unified_Alameda_AR3514.pdf,Fremont_Unified_Alameda_BP3514.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,57726940000000,Yolo,Washington Unified,Urban,Unified School District,8282,67,1,2017,2017,...,0.5,1.50,0.0,6.00,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,environmental safety bp governing board r...,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,ppoolliiccyy eennvviirroonnmmeennttaall ssaaff...,Washington_Unified_Yolo_AR3514.pdf,Washington_Unified_Yolo_BP3514.pdf
700,57727020000000,Yolo,Winters Joint Unified,Town,Unified School District,2279,58,1,2001,2018,...,0.0,0.50,0.0,3.50,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,business noninstructional operation bp concep...,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,business noninstructional operation bp 3000 co...,Winters_Joint_Unified_Yolo_AR3514.pdf,Winters_Joint_Unified_Yolo_BP3514.pdf
701,58727360000000,Yuba,Marysville Joint Unified,Suburban,Unified School District,10539,71,1,2008,2008,...,0.5,1.50,0.0,5.00,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,board education recognizes obligation prov...,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,air quality superintendent designee shall ...,Marysville_Joint_Unified_Yuba_AR3514.pdf,Marysville_Joint_Unified_Yuba_BP3514.pdf
702,58727440000000,Yuba,Plumas Lake Elementary,Town,Elementary School District,1603,45,1,2018,2018,...,0.0,0.50,0.0,4.00,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,board trustee recognizes obligation provi...,/Users/sooyeonkim/Desktop/TenStrands/Policy-La...,plumaslakeelementaryschooldistrict superin...,Plumas_Lake_Elementary_Yuba_AR3514.pdf,Plumas_Lake_Elementary_Yuba_BP3514.pdf


In [25]:
df.to_csv('/Users/sooyeonkim/Desktop/TenStrands/Policy-Language-Analysis/cleaned_data/BP_AR_3514_cleaned.csv', index='False')